In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
!pip install chardet

In [ ]:
#runthis one in case can't read font in file
import chardet

# Read a small part of the file to detect the encoding
with open('IS2_DatasetNorm.csv', 'rb') as f:
    result = chardet.detect(f.read(10000))

# Print the detected encoding
print(result['encoding'])

# Read the CSV file using the detected encoding
df = pd.read_csv('IS2_DatasetNorm.csv', encoding=result['encoding'])

utf-8


In [ ]:
df=pd.read_csv('IS2_DatasetNorm.csv')
df.head()

,Unnamed: 0,Texts,Label,Textclear,TextNorm
0,0,GWM เทลูกค้าที่ซื้อแล้วรถมีปัญหา,"[[""GWM"", ""neg"", ""เทลูกค้า""],[""รถ"", ""neg"", ""มีป...",GWM เทลูกค้าที่ซื้อแล้วรถมีปัญหา,GWM เทลูกค้าที่ซื้อแล้วรถมีปัญหา
1,1,Segment นี้แบรนอื่นรับไฟที่ตู้ DC กันระดับ 120...,"[[""สเปคชาร์จไฟ"", ""neg"", [""ช้า"",""ความน่าสนใจลดล...",Segment นี้แบรนอื่นรับไฟที่ตู้ DC กันระดับ 120...,Segment นี้แบรนอื่นรับไฟที่ตู้ DC กันระดับ 120...
2,2,Software บ้งเหมือนเดิม ไม่คิดจะแก้บ้างเลยหรอ GWM,"[[""Software"", ""neg"", ""บ้งเหมือนเดิม""], [""GWM"",...",Software บ้งเหมือนเดิม ไม่คิดจะแก้บ้างเลยหรอ GWM,Software บ้งเหมือนเดิม ไม่คิดจะแก้บ้างเลยหรอ GWM
3,3,supercarไฟฟ้าชัดๆๆเหลือราคาถ้าราคาดีน่าสน,"[[""รถ"", ""pos"", ""supercarไฟฟ้า""]]",supercarไฟฟ้าชัดๆๆเหลือราคาถ้าราคาดีน่าสน,supercarไฟฟ้าชัดๆๆเหลือราคาถ้าราคาดีน่าสน
4,4,ข้างนอกสวย ข้างในอย่างเชยยย,"[[""ข้างนอก"", ""pos"", ""สวย""],[""ข้างใน"", ""neg"", ""...",ข้างนอกสวย ข้างในอย่างเชยยย,ข้างนอกสวย ข้างในอย่างเชยยย


In [ ]:
sample = df.iloc[:199]

Quantize model to reduce model size

In [ ]:
import IPython
import sys

def clean_notebook():
    IPython.display.clear_output(wait=True)
    print("Notebook cleaned.")

!pip install openai
!pip install bitsandbytes accelerate

# Clean up the notebook
clean_notebook()

Notebook cleaned.


In [ ]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [ ]:
import transformers
from transformers import AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig
import torch

# Load tokenizer and model with QLoRA configuration
# AI model normally 32 bits too big --> need quantile to reduce size - to be able to run the model but trade off with accuracy
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Prompts

In [ ]:
Fewshot_prompt = """You are a text analysis model. Your task is to extract aspects, sentiments, and opinion words from a given sentence that is an opinion about a newly launched car.
Task:
For each aspect in the sentence, you need to identify:
The aspect terms which are related to car (if explicitly present or inferred from context. If not explicitly mentioned,provide the most likely one based on the context).
The sentiment associated with the aspect ('pos' for positive, 'neg' for negative).
The opinion words related to each aspect. (if explicitly present or inferred from context. If not explicitly mentioned,provide the most likely one).
Then return the output in a format of a list of lists format without additional explanation in front or at the end of the list:
[["aspect1", "sentiment1", "opinion_word1"], ["aspect2", "sentiment2", ["opinion_word2.1", "opinion_word2.2",...]], ...]
If a sentence has multiple aspects or opinions, list each aspect separately in the output.
The output should be a list of lists, where each inner list contains: The aspect term, the sentiment and the opinion words
Output only in the format without extra explanation or characters.

Example Outputs:
Input: "ดีไซน์สวยมาก แต่ราคาแพงเกิน"
Output: [["ดีไซน์", "pos", "สวย"], ["ราคา", "neg", "แพงเกิน"]]

Input: "สวย ดุดัน แรง ออปชั่นดี ติดอย่างเดียวเรื่องแบรนด์"
Output: [["ดีไซน์", "pos", ["สวย", "ดุดัน"]], ["เครื่องยนต์", "pos", "แรง"], ["ออปชั่น", "pos", "ดี"], ["แบรนด์", "neg", "ไม่ดี"]]

Input: "ทั้งวัสดุทั้งงานประกอบแย่มาก"
Output: [["วัสดุ", "neg", "แย่"], ["งานประกอบ", "neg", "แย่"]]

Input: "ดูแล้วเหมือนหัวจะชนหลังคาดูอึดอัด"
Output: [["พื้นที่ห้องโดยสาร", "neg", ["เหมือนหัวจะชนหลังคา","ดูอึดอัด"]]]

Input: "ถ้าเตี้ยแบบนี้ ลุยน้ำท่วมไม่รอดแน่"
Output: [["รถ", "neg", ["เตี้ย","ลุยน้ำท่วมไม่รอด"]]]

Input: "รถก็หล่อ พี่นิธิก็เท่ห์ วันนี้ใส่เสื้อฮาวายซะด้วย"
Output: [["รถ", "pos", "หล่อ"]]

Input: "Your sentence here"
Output:
"""

In [ ]:
Oneshot_prompt = """You are a text analysis model. Your task is to extract aspects, sentiments, and opinion words from a given sentence that is an opinion about a newly launched car.
Task:
For each aspect in the sentence, you need to identify:
The aspect terms which are related to car (if explicitly present or inferred from context. If not explicitly mentioned,provide the most likely one based on the context).
The sentiment associated with the aspect ('pos' for positive, 'neg' for negative).
The opinion words related to each aspect. (if explicitly present or inferred from context. If not explicitly mentioned,provide the most likely one).
Then return the output in a format of a list of lists format without additional explanation in front or at the end of the list:
[["aspect1", "sentiment1", "opinion_word1"], ["aspect2", "sentiment2", ["opinion_word2.1", "opinion_word2.2",...]], ...]
If a sentence has multiple aspects or opinions, list each aspect separately in the output.
The output should be a list of lists, where each inner list contains: The aspect term, the sentiment and the opinion words
Output only in the format without extra explanation or characters.

Example Outputs:
Input: "สวย ดุดัน แรง ออปชั่นดี ติดอย่างเดียวเรื่องแบรนด์"
Output: [["ดีไซน์", "pos", ["สวย", "ดุดัน"]], ["เครื่องยนต์", "pos", "แรง"], ["ออปชั่น", "pos", "ดี"], ["แบรนด์", "neg", "ไม่ดี"]]

Input: "Your sentence here"
Output:
"""

In [ ]:
Zeroshot_prompt = """You are a text analysis model. Your task is to extract aspects, sentiments, and opinion words from a given sentence that is an opinion about a newly launched car.
Task:
For each aspect in the sentence, you need to identify:
The aspect terms which are related to car (if explicitly present or inferred from context. If not explicitly mentioned,provide the most likely one based on the context).
The sentiment associated with the aspect ('pos' for positive, 'neg' for negative).
The opinion words related to each aspect. (if explicitly present or inferred from context. If not explicitly mentioned,provide the most likely one).
Then return the output in a format of a list of lists format without additional explanation in front or at the end of the list:
[["aspect1", "sentiment1", "opinion_word1"], ["aspect2", "sentiment2", ["opinion_word2.1", "opinion_word2.2",...]], ...]
If a sentence has multiple aspects or opinions, list each aspect separately in the output.
The output should be a list of lists, where each inner list contains: The aspect term, the sentiment and the opinion words """

In [ ]:
Thai_prompt = """คุณเป็นโมเดลวิเคราะห์ข้อความ งานของคุณคือการดึงข้อมูลด้านต่าง ๆ, อารมณ์ และคำแสดงความคิดเห็น จากประโยคที่เกี่ยวกับรถยนต์รุ่นใหม่
สำหรับแต่ละด้านในประโยค คุณต้องระบุ:
- คำที่เกี่ยวข้องกับรถยนต์ (ถ้ามี หรืออนุมานจากบริบท หากไม่ระบุ ให้เลือกคำที่น่าจะเป็นมากที่สุด)
- อารมณ์ที่เกี่ยวข้อง ('pos' สำหรับเชิงบวก, 'neg' สำหรับเชิงลบ)
- คำแสดงความคิดเห็นที่เกี่ยวข้องกับแต่ละด้าน (ถ้ามี หรืออนุมานจากบริบท)

จากนั้นให้ผลลัพธ์ในรูปแบบรายการที่อยู่ในลิสต์ (list of lists) โดยไม่มีคำอธิบายเพิ่มเติมใด ๆ ก่อนหรือหลังลิสต์:
[["ด้าน1", "อารมณ์1", "คำแสดงความคิดเห็น1"], ["ด้าน2", "อารมณ์2", ["คำแสดงความคิดเห็น2.1", "คำแสดงความคิดเห็น2.2", ...]], ...]

ถ้าประโยคมีหลายด้านหรือความคิดเห็น ให้ระบุแต่ละด้านแยกกันในผลลัพธ์
ผลลัพธ์จะต้องเป็นลิสต์ที่มีลิสต์ย่อย โดยแต่ละลิสต์ย่อยประกอบด้วย: คำที่เกี่ยวข้องกับด้าน อารมณ์ และคำแสดงความคิดเห็นที่เกี่ยวข้อง

ตัวอย่าง:
Input: "ดีไซน์สวยมาก แต่ราคาแพงเกิน"
Output: [["ดีไซน์", "pos", "สวย"], ["ราคา", "neg", "แพงเกิน"]]

Input: "สวย ดุดัน แรง ออปชั่นดี ติดอย่างเดียวเรื่องแบรนด์"
Output: [["ดีไซน์", "pos", ["สวย", "ดุดัน"]], ["เครื่องยนต์", "pos", "แรง"], ["ออปชั่น", "pos", "ดี"], ["แบรนด์", "neg", "ไม่ดี"]]

Input: "ทั้งวัสดุทั้งงานประกอบแย่มาก"
Output: [["วัสดุ", "neg", "แย่"], ["งานประกอบ", "neg", "แย่"]]

Input: "ดูแล้วเหมือนหัวจะชนหลังคาดูอึดอัด"
Output: [["พื้นที่ห้องโดยสาร", "neg", ["เหมือนหัวจะชนหลังคา","ดูอึดอัด"]]]

Input: "ถ้าเตี้ยแบบนี้ ลุยน้ำท่วมไม่รอดแน่"
Output: [["รถ", "neg", ["เตี้ย","ลุยน้ำท่วมไม่รอด"]]]

Input: "รถก็หล่อ พี่นิธิก็เท่ห์ วันนี้ใส่เสื้อฮาวายซะด้วย"
Output: [["รถ", "pos", "หล่อ"]]

Input: "ประโยคของคุณที่นี่"
Output:
"""

# Parameters

In [ ]:
max_tokens= 256  #the best comparing with 128 & 512
temperature=0.4  # the best among 0.1-0.5
top_p=0.9

**Check the best Temperature**

In [ ]:
#Huggingface API
import os
os.environ['HF_TOKEN'] ="Your API Key"
hf_token               = os.environ['HF_TOKEN']

In [ ]:
Lmodel_id = "meta-llama/Meta-Llama-3-8B-Instruct"  # as Benchmark model

Lmodel     = AutoModelForCausalLM.from_pretrained(Lmodel_id, quantization_config=bnb_config,device_map="auto",token=hf_token)
Ltokenizer = AutoTokenizer.from_pretrained(Lmodel_id)
Ltokenizer.pad_token = Ltokenizer.eos_token

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [ ]:
Lmodel

In [ ]:
def generate_model(model, tokenizer, prompt, system_prompt, temperature, max_tokens=max_tokens,top_p=top_p):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ]

    # Tokenize with attention mask
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        padding=True,
        truncation=True,
        return_tensors="pt"
    ).to(model.device)

    if isinstance(inputs, dict): #check if inputs are dict
        input_ids = inputs["input_ids"]
        attention_mask = inputs["attention_mask"]
    else:
        input_ids = inputs
        attention_mask = (input_ids != tokenizer.pad_token_id).long()  # Create attention mask

    # Set pad_token_id explicitly if needed
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id

    # Generate the output
    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=max_tokens,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
    )

    # Decode the response, excluding the prompt tokens
    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)

check the best temperature parameter

In [ ]:
def process_llama01(sentence):  #repeat this step for temperature 0.2 - 0.5
    result = generate_model(Lmodel,Ltokenizer,sentence,Zeroshot_prompt,temperature=0.1)
    return result

In [ ]:
Sample = df.iloc[0:199] # 10% of data
Sample['zerotemp0.1'] = Sample['TextNorm'].apply(process_llama01)
Sample.to_csv('Sample_llama0shot_temp.csv')

Check Max_tokens parameter

In [ ]:
def process_llama256(sentence):
    result = generate_model(Lmodel,Ltokenizer,sentence,Fewshot_prompt,temperature)
    return result

In [ ]:
df['Llama256_result'] = df['TextNorm'].apply(process_llama256)
df.to_csv('/content/drive/My Drive/Llama256_result.csv')
df.head()

,Unnamed: 0,Texts,Label,Textclear,TextNorm,Llama256_result
0,0,GWM เทลูกค้าที่ซื้อแล้วรถมีปัญหา,"[[""GWM"", ""neg"", ""เทลูกค้า""],[""รถ"", ""neg"", ""มีป...",GWM เทลูกค้าที่ซื้อแล้วรถมีปัญหา,GWM เทลูกค้าที่ซื้อแล้วรถมีปัญหา,"[[""คุณภาพ"", ""neg"", ""มีปัญหา""]]"
1,1,Segment นี้แบรนอื่นรับไฟที่ตู้ DC กันระดับ 120...,"[[""สเปคชาร์จไฟ"", ""neg"", [""ช้า"",""ความน่าสนใจลดล...",Segment นี้แบรนอื่นรับไฟที่ตู้ DC กันระดับ 120...,Segment นี้แบรนอื่นรับไฟที่ตู้ DC กันระดับ 120...,"[[""สเปคชาร์จไฟ"", ""neg"", [""ช้าๆ""]], [""ความน่าสน..."
2,2,Software บ้งเหมือนเดิม ไม่คิดจะแก้บ้างเลยหรอ GWM,"[[""Software"", ""neg"", ""บ้งเหมือนเดิม""], [""GWM"",...",Software บ้งเหมือนเดิม ไม่คิดจะแก้บ้างเลยหรอ GWM,Software บ้งเหมือนเดิม ไม่คิดจะแก้บ้างเลยหรอ GWM,"[[""Software"", ""neg"", [""บ้งเหมือนเดิม"", ""ไม่คิด..."
3,3,supercarไฟฟ้าชัดๆๆเหลือราคาถ้าราคาดีน่าสน,"[[""รถ"", ""pos"", ""supercarไฟฟ้า""]]",supercarไฟฟ้าชัดๆๆเหลือราคาถ้าราคาดีน่าสน,supercarไฟฟ้าชัดๆๆเหลือราคาถ้าราคาดีน่าสน,"[[""รถ"", ""pos"", [""ไฟฟ้าชัดๆๆ""]], [""ราคา"", ""neg""..."
4,4,ข้างนอกสวย ข้างในอย่างเชยยย,"[[""ข้างนอก"", ""pos"", ""สวย""],[""ข้างใน"", ""neg"", ""...",ข้างนอกสวย ข้างในอย่างเชยยย,ข้างนอกสวย ข้างในอย่างเชยยย,"[[""ดีไซน์"", ""pos"", [""สวย""]], [""ภายใน"", ""neg"", ..."


In [ ]:
df.to_csv('Llama256_result.csv')

In [ ]:
def process_llama128(sentence):
    result = generate_model(Lmodel,Ltokenizer,sentence,Fewshot_prompt,temperature)
    return result

In [ ]:
df['Llama128_result'] = df['TextNorm'].apply(process_llama128)
df.to_csv('/content/drive/My Drive/Llama128_result.csv')
df.to_csv('Llama128_result.csv')
df.head()

In [ ]:
def process_llama512(sentence):
    result = generate_model(Lmodel,Ltokenizer,sentence,Fewshot_prompt,temperature)
    return result

In [ ]:
df['Llama512_result'] = df['TextNorm'].apply(process_llama512)
df.to_csv('/content/drive/My Drive/Llama512_result.csv')
df.to_csv('Llama512_result.csv')
df.head()

Check Thai prompt

In [ ]:
def process_llamaTH(sentence):
    result = generate_model(Lmodel,Ltokenizer,sentence,Thai_prompt,temperature=0.4)
    return result

sample['Llama_Thaiprompt'] = sample['TextNorm'].apply(process_llamaTH)
sample.to_csv('Sample_llama_THprompt.csv', index=False, encoding='utf-8-sig')

<ipython-input-16-a7c9a58bc486>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['Llama_Thaiprompt'] = sample['TextNorm'].apply(process_llamaTH)


# Check the best Prompt type

In [ ]:
def process_zero(sentence):
    result = generate_model(Lmodel,Ltokenizer,sentence,Zeroshot_prompt,temperature=temperature)
    return result

df['Zeroshot_llama'] = df['TextNorm'].apply(process_zero)
print(df.head())
df.to_csv('Zeroshot_llama_result.csv', index=False)

In [ ]:
def process_one(sentence):
    result = generate_model(Lmodel,Ltokenizer,sentence,Oneshot_prompt,temperature=temperature)
    return result

df['Oneshot_llama'] = df['TextNorm'].apply(process_one)
print(df.head())
df.to_csv('Oneshot_llama_result.csv', index=False)

In [ ]:
def process_few(sentence):
    result = generate_model(Lmodel,Ltokenizer,sentence,Fewshot_prompt,temperature=temperature)
    return result

df['Fewshot_llama'] = df['TextNorm'].apply(process_few)
print(df.head())
df.to_csv('Fewshot_llama_result.csv', index=False)
# use this result for Llama3 model comparison

Then check performance in F1 Evaluation file

# Model comparison

**GPT 3.5 Turbo & GPT 4o mini**


In [ ]:
import openai
openai.api_key = "Your API Key"

In [ ]:
def complete(model, user_prompt, system_prompt):
    completion = openai.chat.completions.create(
        model= model,  #gpt-3.5-turbo, gpt-4o-mini
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.4,
        max_tokens=256,
        top_p=0.9,
    )
    return completion.choices[0].message.content

In [ ]:
# GPT 3.5 Turbo for annotation
def process_GPT35(sentence):
    system_prompt = Fewshot_prompt
    result = complete("gpt-3.5-turbo", sentence, system_prompt)
    return result


# Apply the function to dataset
df['GPT3.5 turbo_analysis'] = df['TextNorm'].apply(process_GPT35)
print(df.head())
df.to_csv('GPT3.5 turbo_result.csv', index=False)

In [ ]:
# GPT 4o mini
def process_GPT4o(sentence):
    system_prompt = Fewshot_prompt
    result = complete("gpt-4o-mini", sentence, system_prompt)
    return result


# Apply the function to dataset
df['GPT4o mini_analysis'] = df['TextNorm'].apply(process_GPT4o)
print(df.head())
df.to_csv('GPT4o mini_result.csv', index=False, encoding='utf-8-sig')


   Unnamed: 0                                              Texts  \
0           0                   GWM เทลูกค้าที่ซื้อแล้วรถมีปัญหา   
1           1  Segment นี้แบรนอื่นรับไฟที่ตู้ DC กันระดับ 120...   
2           2   Software บ้งเหมือนเดิม ไม่คิดจะแก้บ้างเลยหรอ GWM   
3           3          supercarไฟฟ้าชัดๆๆเหลือราคาถ้าราคาดีน่าสน   
4           4                        ข้างนอกสวย ข้างในอย่างเชยยย   

                                               Label  \
0  [["GWM", "neg", "เทลูกค้า"],["รถ", "neg", "มีป...   
1  [["สเปคชาร์จไฟ", "neg", ["ช้า","ความน่าสนใจลดล...   
2  [["Software", "neg", "บ้งเหมือนเดิม"], ["GWM",...   
3                   [["รถ", "pos", "supercarไฟฟ้า"]]   
4  [["ข้างนอก", "pos", "สวย"],["ข้างใน", "neg", "...   

                                           Textclear  \
0                   GWM เทลูกค้าที่ซื้อแล้วรถมีปัญหา   
1  Segment นี้แบรนอื่นรับไฟที่ตู้ DC กันระดับ 120...   
2   Software บ้งเหมือนเดิม ไม่คิดจะแก้บ้างเลยหรอ GWM   
3          supercarไฟฟ้าชัดๆๆเ

**Huggingface models:**\
    **Typhoon**

In [ ]:
#Typhoon
Tmodel_id = "scb10x/llama-3-typhoon-v1.5-8b-instruct"  #scb10x/typhoon-7b-instruct-02-19-2024

Tmodel     = AutoModelForCausalLM.from_pretrained(Tmodel_id, quantization_config=bnb_config,device_map="auto",token=hf_token)
Ttokenizer = AutoTokenizer.from_pretrained(Tmodel_id)

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

In [ ]:
def process_typhoon256(sentence):  #repeat this step for temperature 0.2 - 0.5
    result = generate_model(Tmodel,Ttokenizer,sentence,Fewshot_prompt,temperature)
    return result

In [ ]:
df['Typhoon256_result'] = df['TextNorm'].apply(process_typhoon256)
print(df.head())

df.to_csv('Typhoon256.csv', index=False, encoding='utf-8-sig')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


   Unnamed: 0                                              Texts  \
0           0                   GWM เทลูกค้าที่ซื้อแล้วรถมีปัญหา   
1           1  Segment นี้แบรนอื่นรับไฟที่ตู้ DC กันระดับ 120...   
2           2   Software บ้งเหมือนเดิม ไม่คิดจะแก้บ้างเลยหรอ GWM   
3           3          supercarไฟฟ้าชัดๆๆเหลือราคาถ้าราคาดีน่าสน   
4           4                        ข้างนอกสวย ข้างในอย่างเชยยย   

                                               Label  \
0  [["GWM", "neg", "เทลูกค้า"],["รถ", "neg", "มีป...   
1  [["สเปคชาร์จไฟ", "neg", ["ช้า","ความน่าสนใจลดล...   
2  [["Software", "neg", "บ้งเหมือนเดิม"], ["GWM",...   
3                   [["รถ", "pos", "supercarไฟฟ้า"]]   
4  [["ข้างนอก", "pos", "สวย"],["ข้างใน", "neg", "...   

                                           Textclear  \
0                   GWM เทลูกค้าที่ซื้อแล้วรถมีปัญหา   
1  Segment นี้แบรนอื่นรับไฟที่ตู้ DC กันระดับ 120...   
2   Software บ้งเหมือนเดิม ไม่คิดจะแก้บ้างเลยหรอ GWM   
3          supercarไฟฟ้าชัดๆๆเ

**WangchanX**

In [ ]:
WXmodel_id = "airesearch/LLaMa3-8b-WangchanX-sft-Full"

WXmodel     = AutoModelForCausalLM.from_pretrained(WXmodel_id, quantization_config=bnb_config,device_map="auto",token=hf_token)
WXtokenizer = AutoTokenizer.from_pretrained(WXmodel_id)

config.json:   0%|          | 0.00/697 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00007.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00003-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

In [ ]:
def process_WangchanX256(sentence):  #repeat this step for temperature 0.2 - 0.5
    result = generate_model(WXmodel,WXtokenizer,sentence,Fewshot_prompt,temperature)
    return result

In [ ]:
df['WangchanX256_result'] = df['TextNorm'].apply(process_WangchanX256)
print(df.head())

df.to_csv('WangchanX256.csv', index=False, encoding='utf-8-sig')
df.to_csv('/content/drive/My Drive/WangchanX256_result.csv', index=False, encoding='utf-8-sig')

   Unnamed: 0                                              Texts  \
0           0                   GWM เทลูกค้าที่ซื้อแล้วรถมีปัญหา   
1           1  Segment นี้แบรนอื่นรับไฟที่ตู้ DC กันระดับ 120...   
2           2   Software บ้งเหมือนเดิม ไม่คิดจะแก้บ้างเลยหรอ GWM   
3           3          supercarไฟฟ้าชัดๆๆเหลือราคาถ้าราคาดีน่าสน   
4           4                        ข้างนอกสวย ข้างในอย่างเชยยย   

                                               Label  \
0  [["GWM", "neg", "เทลูกค้า"],["รถ", "neg", "มีป...   
1  [["สเปคชาร์จไฟ", "neg", ["ช้า","ความน่าสนใจลดล...   
2  [["Software", "neg", "บ้งเหมือนเดิม"], ["GWM",...   
3                   [["รถ", "pos", "supercarไฟฟ้า"]]   
4  [["ข้างนอก", "pos", "สวย"],["ข้างใน", "neg", "...   

                                           Textclear  \
0                   GWM เทลูกค้าที่ซื้อแล้วรถมีปัญหา   
1  Segment นี้แบรนอื่นรับไฟที่ตู้ DC กันระดับ 120...   
2   Software บ้งเหมือนเดิม ไม่คิดจะแก้บ้างเลยหรอ GWM   
3          supercarไฟฟ้าชัดๆๆเ

OSError: Cannot save file into a non-existent directory: '/content/drive/My Drive'

**SEA-LION**


In [ ]:
Smodel_id = "aisingapore/llama3-8b-cpt-sealionv2-instruct"
Smodel     = AutoModelForCausalLM.from_pretrained(Smodel_id, quantization_config=bnb_config,device_map="auto",token=hf_token)
Stokenizer = AutoTokenizer.from_pretrained(Smodel_id)

config.json:   0%|          | 0.00/703 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

In [ ]:
def process_sealion256(sentence):  #repeat this step for temperature 0.2 - 0.5
    result = generate_model(Smodel,Stokenizer,sentence,Fewshot_prompt,temperature)
    return result

In [ ]:
df['Sealion256_result'] = df['TextNorm'].apply(process_sealion256)
print(df.head())

df.to_csv('Sealion256.csv', index=False, encoding='utf-8-sig')
df.to_csv('/content/drive/My Drive/Sealion256_result.csv', index=False, encoding='utf-8-sig')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


   Unnamed: 0                                              Texts  \
0           0                   GWM เทลูกค้าที่ซื้อแล้วรถมีปัญหา   
1           1  Segment นี้แบรนอื่นรับไฟที่ตู้ DC กันระดับ 120...   
2           2   Software บ้งเหมือนเดิม ไม่คิดจะแก้บ้างเลยหรอ GWM   
3           3          supercarไฟฟ้าชัดๆๆเหลือราคาถ้าราคาดีน่าสน   
4           4                        ข้างนอกสวย ข้างในอย่างเชยยย   

                                               Label  \
0  [["GWM", "neg", "เทลูกค้า"],["รถ", "neg", "มีป...   
1  [["สเปคชาร์จไฟ", "neg", ["ช้า","ความน่าสนใจลดล...   
2  [["Software", "neg", "บ้งเหมือนเดิม"], ["GWM",...   
3                   [["รถ", "pos", "supercarไฟฟ้า"]]   
4  [["ข้างนอก", "pos", "สวย"],["ข้างใน", "neg", "...   

                                           Textclear  \
0                   GWM เทลูกค้าที่ซื้อแล้วรถมีปัญหา   
1  Segment นี้แบรนอื่นรับไฟที่ตู้ DC กันระดับ 120...   
2   Software บ้งเหมือนเดิม ไม่คิดจะแก้บ้างเลยหรอ GWM   
3          supercarไฟฟ้าชัดๆๆเ

**OpenThaiGPT**

In [ ]:
OTmodel_id = "openthaigpt/openthaigpt1.5-7b-instruct"

OTmodel = AutoModelForCausalLM.from_pretrained(OTmodel_id, quantization_config=bnb_config,device_map='auto',token=hf_token)
OTtokenizer = AutoTokenizer.from_pretrained(OTmodel_id)
OTtokenizer.pad_token = OTtokenizer.eos_token

config.json:   0%|          | 0.00/692 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
def process_thaigpt(sentence):  #repeat this step for temperature 0.2 - 0.5
    result = generate_model(OTmodel,OTtokenizer,sentence,Fewshot_prompt,temperature)
    return result

In [ ]:
df['ThaiGPT_result'] = df['TextNorm'].apply(process_thaigpt)
df.to_csv('/content/drive/My Drive/ThaiGPT256_result.csv', index=False, encoding='utf-8-sig')
df.head()

,Unnamed: 0,Texts,Label,Textclear,TextNorm,ThaiGPT_result
0,0,GWM เทลูกค้าที่ซื้อแล้วรถมีปัญหา,"[[""GWM"", ""neg"", ""เทลูกค้า""],[""รถ"", ""neg"", ""มีป...",GWM เทลูกค้าที่ซื้อแล้วรถมีปัญหา,GWM เทลูกค้าที่ซื้อแล้วรถมีปัญหา,"[""GWM"", ""neg"", ""เทลูกค้า""]"
1,1,Segment นี้แบรนอื่นรับไฟที่ตู้ DC กันระดับ 120...,"[[""สเปคชาร์จไฟ"", ""neg"", [""ช้า"",""ความน่าสนใจลดล...",Segment นี้แบรนอื่นรับไฟที่ตู้ DC กันระดับ 120...,Segment นี้แบรนอื่นรับไฟที่ตู้ DC กันระดับ 120...,"[[""แบรนด์"", ""neg"", [""รับไฟที่ตู้ DC"", ""ชาร์จไฟ..."
2,2,Software บ้งเหมือนเดิม ไม่คิดจะแก้บ้างเลยหรอ GWM,"[[""Software"", ""neg"", ""บ้งเหมือนเดิม""], [""GWM"",...",Software บ้งเหมือนเดิม ไม่คิดจะแก้บ้างเลยหรอ GWM,Software บ้งเหมือนเดิม ไม่คิดจะแก้บ้างเลยหรอ GWM,"[""ซอฟต์แวร์"", ""neg"", [""บ้งเหมือนเดิม"", ""ไม่คิด..."
3,3,supercarไฟฟ้าชัดๆๆเหลือราคาถ้าราคาดีน่าสน,"[[""รถ"", ""pos"", ""supercarไฟฟ้า""]]",supercarไฟฟ้าชัดๆๆเหลือราคาถ้าราคาดีน่าสน,supercarไฟฟ้าชัดๆๆเหลือราคาถ้าราคาดีน่าสน,"[[""supercar"", ""pos"", ""ไฟฟ้าชัด""], [""ราคา"", ""po..."
4,4,ข้างนอกสวย ข้างในอย่างเชยยย,"[[""ข้างนอก"", ""pos"", ""สวย""],[""ข้างใน"", ""neg"", ""...",ข้างนอกสวย ข้างในอย่างเชยยย,ข้างนอกสวย ข้างในอย่างเชยยย,"[[""ดีไซน์"", ""pos"", ""สวย""], [""ภายใน"", ""neg"", ""อ..."


In [ ]:
df.to_csv('ThaiGPT256.csv', index=False, encoding='utf-8-sig')